In [20]:
import glob
import datetime as dt
from datetime import datetime, timedelta
import pandas as pd
from dateutil.relativedelta import *
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
from sklearn.metrics import mean_absolute_error
from fbprophet import Prophet

In [21]:
name = 'Price_fullset.csv'
data = pd.read_csv(name, index_col=False)
data['Timestamp']= pd.to_datetime(data['Timestamp'])
data = data[['Timestamp','Region','Total','Energy','Lost','Congestion']]
pivot = data.pivot_table('Total', ['Timestamp'], 'Region')
pivot = pivot[['MONTERREY']]
pivot = pivot.reset_index(drop = False)
pivot.to_csv('Monterrey_historical_data.csv', index = False)
pivot

Region,Timestamp,MONTERREY
0,2016-01-29 00:00:00,744.88
1,2016-01-29 01:00:00,560.36
2,2016-01-29 02:00:00,497.98
3,2016-01-29 03:00:00,466.42
4,2016-01-29 04:00:00,484.61
...,...,...
40982,2020-10-01 19:00:00,450.56
40983,2020-10-01 20:00:00,486.42
40984,2020-10-01 21:00:00,458.14
40985,2020-10-01 22:00:00,458.90


In [22]:
name = 'Monterrey_historical_data.csv'
data = pd.read_csv(name, index_col=False)
data

,Timestamp,MONTERREY
0,2016-01-29 00:00:00,744.88
1,2016-01-29 01:00:00,560.36
2,2016-01-29 02:00:00,497.98
3,2016-01-29 03:00:00,466.42
4,2016-01-29 04:00:00,484.61
...,...,...
40982,2020-10-01 19:00:00,450.56
40983,2020-10-01 20:00:00,486.42
40984,2020-10-01 21:00:00,458.14
40985,2020-10-01 22:00:00,458.90


In [23]:
data['MONTERREY'].isna().sum()

0

In [29]:
prophet_pred = prophet_prediction('2016-01-29 00:00:00','2020-08-31 23:00:00','2020-09-01 00:00:00',\
                   '2020-09-07 23:00:00',data,'1h')
prophet_pred = prophet_pred[['ds','yhat']]
prophet_pred

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


,ds,yhat
0,2020-09-01 00:00:00,667.572959
1,2020-09-01 01:00:00,587.774021
2,2020-09-01 02:00:00,522.236678
3,2020-09-01 03:00:00,484.164425
4,2020-09-01 04:00:00,478.151461
...,...,...
163,2020-09-07 19:00:00,739.593884
164,2020-09-07 20:00:00,746.078288
165,2020-09-07 21:00:00,749.358224
166,2020-09-07 22:00:00,730.664455


In [30]:
data_test = data.set_index('Timestamp')
data_test = data_test['2020-09-01 00:00:00':'2020-09-07 23:00:00']
data_test = data_test.reset_index(drop = False)
data_test

,Timestamp,MONTERREY
0,2020-09-01 00:00:00,637.03
1,2020-09-01 01:00:00,630.91
2,2020-09-01 02:00:00,607.61
3,2020-09-01 03:00:00,593.64
4,2020-09-01 04:00:00,582.30
...,...,...
163,2020-09-07 19:00:00,540.69
164,2020-09-07 20:00:00,565.71
165,2020-09-07 21:00:00,571.54
166,2020-09-07 22:00:00,566.56


In [57]:
mean_absolute_error(data_test['MONTERREY'], prophet_pred['yhat'])

148.5443435929889

In [58]:
mean_absolute_percentage_error(data_test['MONTERREY'], prophet_pred['yhat'])

26.172862827623288

In [59]:
(data_test['MONTERREY']-prophet_pred['yhat']).sum()/len(data_test['MONTERREY'])

-4.570714987284922

# ARIMA

In [34]:
name = 'Monterrey_historical_data.csv'
data = pd.read_csv(name, index_col=False)
data = data.set_index('Timestamp')
data = data['2016-01-29 00:00:00':'2020-08-31 23:00:00']
data = data.reset_index(drop = False)
data['Timestamp'] = pd.DatetimeIndex(data['Timestamp'])
data = data.set_index('Timestamp')
data

,MONTERREY
Timestamp,
2016-01-29 00:00:00,744.88
2016-01-29 01:00:00,560.36
2016-01-29 02:00:00,497.98
2016-01-29 03:00:00,466.42
2016-01-29 04:00:00,484.61
...,...
2020-08-31 19:00:00,598.52
2020-08-31 20:00:00,670.66
2020-08-31 21:00:00,701.32


In [35]:
import pmdarima as pm

In [36]:
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel

In [37]:
def predictarima(n_periods, timeseries, automodel):
    # Forecast
    fc, confint = automodel.predict(n_periods=n_periods, 
                                    return_conf_int=True)
    # Weekly index
    fc_ind = pd.date_range(timeseries.index[timeseries.shape[0]-1], 
                           periods=n_periods, freq="H")
    # Forecast series
    fc_series = pd.Series(fc, index=fc_ind)
    return fc_series

In [40]:
automodel = arimamodel(data)
prediction = predictarima(169, data, automodel)

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=542980.888, Time=1.74 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=680501.581, Time=0.67 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=543283.005, Time=0.86 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=635423.594, Time=5.44 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=542981.906, Time=1.28 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=542982.551, Time=2.46 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=606684.463, Time=13.70 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=542979.907, Time=1.07 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=542981.906, Time=1.34 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=542982.534, Time=4.27 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=542287.933, Time=3.41 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=542474.435, Time=2.80 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=542277.598, Time=4.63 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=542277.280, Time=7.52 sec
 ARI

In [44]:
name = 'Monterrey_historical_data.csv'
data1 = pd.read_csv(name, index_col=False)
data_test = data1.set_index('Timestamp')
data_test = data_test['2020-09-01 00:00:00':'2020-09-07 23:00:00']
data_test = data_test.reset_index(drop = False)
data_test

,Timestamp,MONTERREY
0,2020-09-01 00:00:00,637.03
1,2020-09-01 01:00:00,630.91
2,2020-09-01 02:00:00,607.61
3,2020-09-01 03:00:00,593.64
4,2020-09-01 04:00:00,582.30
...,...,...
163,2020-09-07 19:00:00,540.69
164,2020-09-07 20:00:00,565.71
165,2020-09-07 21:00:00,571.54
166,2020-09-07 22:00:00,566.56


In [49]:
prediction = predictarima(169, data, automodel)

In [50]:
prediction = prediction.reset_index(drop = False)
prediction.columns = ['date','prediction']
prediction = prediction[1:].reset_index(drop = True)
prediction

,date,prediction
0,2020-09-01 00:00:00,729.830705
1,2020-09-01 01:00:00,740.951657
2,2020-09-01 02:00:00,751.412392
3,2020-09-01 03:00:00,761.290476
4,2020-09-01 04:00:00,770.686698
...,...,...
163,2020-09-07 19:00:00,953.248967
164,2020-09-07 20:00:00,953.252038
165,2020-09-07 21:00:00,953.254958
166,2020-09-07 22:00:00,953.257735


In [60]:
mean_absolute_error(data_test['MONTERREY'],prediction['prediction'])

335.8098921385778

In [61]:
mean_absolute_percentage_error(data_test['MONTERREY'],prediction['prediction'])

60.12047389437862

In [62]:
(data_test['MONTERREY']-prediction['prediction']).sum()/len(data_test['MONTERREY'])

-335.8098921385778

# ESRNN

In [72]:
from ESRNN.m4_data import prepare_m4_data
from ESRNN.utils_evaluation import evaluate_prediction_owa

from ESRNN import ESRNN


In [74]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name='Yearly',
                                                               directory = './data',
                                                               num_obs=1000)

Successfully downloaded M4-info.csv 4335598 bytes.
Successfully downloaded Train/Daily-train.csv 95765153 bytes.
Successfully downloaded Train/Hourly-train.csv 2347115 bytes.
Successfully downloaded Train/Monthly-train.csv 91655432 bytes.
Successfully downloaded Train/Quarterly-train.csv 38788547 bytes.
Successfully downloaded Train/Weekly-train.csv 4015067 bytes.
Successfully downloaded Train/Yearly-train.csv 25355736 bytes.
Successfully downloaded Test/Daily-test.csv 576459 bytes.
Successfully downloaded Test/Hourly-test.csv 132820 bytes.
Successfully downloaded Test/Monthly-test.csv 7942698 bytes.
Successfully downloaded Test/Quarterly-test.csv 1971754 bytes.
Successfully downloaded Test/Weekly-test.csv 44247 bytes.
Successfully downloaded Test/Yearly-test.csv 1486434 bytes.


Preparing Yearly dataset
Preparing Naive2 Yearly dataset predictions


In [121]:
type(X_train_df['x'][0])

str

In [96]:
X_train_df[-21:]

,unique_id,ds,x
34860,Y998,1970-01-20,Macro
34861,Y999,1970-01-01,Macro
34862,Y999,1970-01-02,Macro
34863,Y999,1970-01-03,Macro
34864,Y999,1970-01-04,Macro
34865,Y999,1970-01-05,Macro
34866,Y999,1970-01-06,Macro
34867,Y999,1970-01-07,Macro
34868,Y999,1970-01-08,Macro
34869,Y999,1970-01-09,Macro


In [90]:
y_train_df

,unique_id,ds,y
0,Y1,1970-01-01,5172.1
1,Y1,1970-01-02,5133.5
2,Y1,1970-01-03,5186.9
3,Y1,1970-01-04,5084.6
4,Y1,1970-01-05,5182.0
...,...,...,...
34876,Y999,1970-01-16,1742.0
34877,Y999,1970-01-17,1420.0
34878,Y999,1970-01-18,971.0
34879,Y999,1970-01-19,563.0


In [91]:
X_test_df

,unique_id,ds,x
0,Y1,1970-02-01,Macro
1,Y1,1970-02-02,Macro
2,Y1,1970-02-03,Macro
3,Y1,1970-02-04,Macro
4,Y1,1970-02-05,Macro
...,...,...,...
5995,Y999,1970-01-22,Macro
5996,Y999,1970-01-23,Macro
5997,Y999,1970-01-24,Macro
5998,Y999,1970-01-25,Macro


In [122]:
# type(y_test_df['unique_id'][0])
y_test_df

,unique_id,ds,y,y_hat_naive2
0,Y1,1970-02-01,7290.2,7261.1
1,Y1,1970-02-02,7392.6,7261.1
2,Y1,1970-02-03,7527.6,7261.1
3,Y1,1970-02-04,7594.8,7261.1
4,Y1,1970-02-05,7720.7,7261.1
...,...,...,...,...
5995,Y999,1970-01-22,469.0,383.0
5996,Y999,1970-01-23,328.0,383.0
5997,Y999,1970-01-24,483.0,383.0
5998,Y999,1970-01-25,437.0,383.0


In [81]:
# Instantiate model
model = ESRNN(max_epochs=25, freq_of_test=5, batch_size=4, learning_rate=1e-4,
              per_series_lr_multip=0.8, lr_scheduler_step_size=10,
              lr_decay=0.1, gradient_clipping_threshold=50,
              rnn_weight_decay=0.0, level_variability_penalty=100,
              testing_percentile=50, training_percentile=50,
              ensemble=False, max_periods=25, seasonality=[],
              input_size=4, output_size=6,
              cell_type='LSTM', state_hsize=40,
              dilations=[[1], [6]], add_nl_layer=False,
              random_seed=1, device='cpu')

# Fit model
# If y_test_df is provided the model
# will evaluate predictions on
# this set every freq_test epochs
model.fit(X_train_df, y_train_df, X_test_df, y_test_df)

# Predict on test set
y_hat_df = model.predict(X_test_df)

# Evaluate predictions
final_owa, final_mase, final_smape = evaluate_prediction_owa(y_hat_df, y_train_df,
                                                             X_test_df, y_test_df,
                                                             naive2_seasonality=1)

Infered frequency: D
=============== Training ESRNN  ===============

========= Epoch 0 finished =========
Training time: 10.74864
Training loss (50 prc): 1.08207
Testing loss  (50 prc): 0.06051
OWA: 0.984 
SMAPE: 13.94 
MASE: 3.87 
========= Epoch 1 finished =========
Training time: 9.39753
Training loss (50 prc): 1.07865
========= Epoch 2 finished =========
Training time: 9.19069
Training loss (50 prc): 1.04170
========= Epoch 3 finished =========
Training time: 11.93237
Training loss (50 prc): 1.05056
========= Epoch 4 finished =========
Training time: 10.11257
Training loss (50 prc): 1.06261
========= Epoch 5 finished =========
Training time: 10.58813
Training loss (50 prc): 1.06209
Testing loss  (50 prc): 0.05886
OWA: 0.92 
SMAPE: 13.11 
MASE: 3.594 
========= Epoch 6 finished =========
Training time: 12.24222
Training loss (50 prc): 1.06603
========= Epoch 7 finished =========
Training time: 11.2606
Training loss (50 prc): 1.04888
========= Epoch 8 finished =========
Training tim

# Example

In [119]:
name = 'Monterrey_historical_data.csv'
data = pd.read_csv(name, index_col=False)
data['unique_id'] = 'Y1' 
# data['ds'] = pd.date_range('2016-01-29 00:00:00', '2020-10-01 23:00:00', freq='1h')
# data['Timestamp'] = pd.DatetimeIndex(data['Timestamp'])

NameError: name 'duration' is not defined

In [120]:
right_list = pd.date_range('2016-01-29 00:00:00', '2020-10-01 23:00:00', freq='H').strftime('%Y-%m-%d %H:%M:%S')
set(right_list) - set(data['Timestamp'])

{'2016-04-03 02:00:00',
 '2017-04-02 02:00:00',
 '2018-04-01 23:00:00',
 '2019-04-07 23:00:00',
 '2020-04-05 23:00:00'}

# Functions

In [25]:
def prophet_prediction(start_train, end_train, start_test, end_test, data, frequen):
#     data = pd.read_csv("demanda_15min2.csv")
#     data = data.reset_index(drop=True)
    
    train_data = partitioning(data, start_train, end_train)
    
    my_model = Prophet(interval_width=0.95)
    my_model.fit(train_data)
    if frequen == '15min':
        future_dates = my_model.make_future_dataframe(periods=2976, freq=frequen)
        
    elif frequen == '30min':
        future_dates = my_model.make_future_dataframe(periods=1488, freq=frequen)
        
    elif frequen == '1h':
        future_dates = my_model.make_future_dataframe(periods=744, freq=frequen)
        
    forecast = my_model.predict(future_dates)
    forecast1 = Re_partitioning(forecast, start_test,end_test)
    
    return forecast1

In [26]:
def partitioning(dataset, start_date, end_date): # choose what start and end date you want for you dataframe 
     principio =   int(dataset[dataset['Timestamp']== start_date].index.values[0])
    
     fin = int(dataset[dataset['Timestamp']== end_date].index.values[0])+1
     
     dataset = dataset[principio:fin].reset_index(drop=True)
     
     dataset['Timestamp'] = pd.DatetimeIndex(dataset['Timestamp'])#prophet readable datetime format

     
     dataset = dataset.rename(columns={'Timestamp': 'ds','MONTERREY': 'y'})
     
     dataset['week#'] = dataset['ds'].dt.week
     
     dataset['day#'] = dataset['ds'].dt.day
     
     dataset['dayofweek#'] = dataset['ds'].dt.dayofweek
     
     dataset['dayofyear#'] = dataset['ds'].dt.dayofyear
    
     
    
     return dataset

In [27]:
def Re_partitioning(dataset, start_date, end_date): # choose what start and end date you want for you dataframe 
     principio =   int(dataset[dataset['ds']== start_date].index.values[0])
    
     fin = int(dataset[dataset['ds']== end_date].index.values[0])+1
     
     dataset = dataset[principio:fin].reset_index(drop=True)
     
     dataset['ds'] = pd.DatetimeIndex(dataset['ds'])#prophet readable datetime format

     
     #dataset = dataset.rename(columns={'DATE_TIME': 'ds','DEMAND': 'y'})
     
     dataset['week#'] = dataset['ds'].dt.week
    
     dataset['day#'] = dataset['ds'].dt.day
     
     dataset['dayofweek#'] = dataset['ds'].dt.dayofweek
    
     dataset['dayofyear#'] = dataset['ds'].dt.dayofyear
     
    
     return dataset

In [28]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100